## 東北大BERTをベースにファインチューニングで固有表現抽出用モデルを作成する
huggingfaceで公開されている東北大BERTこと `cl-tohoku/bert-base-japanese-whole-word-masking` をベースに、ファインチューニングをして固有表現抽出タスク用のモデルを作成します

## 準備

### ライブラリのインストール
必要なライブラリをインストールします

In [ ]:
!pip3 install --upgrade pip
!pip3 install transformers["ja"] numpy noyaki sklearn seqeval
!pip3 install -U jupyter ipywidgets
!pip3 install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

### 学習データのダウンロード
今回は[ストックマーク株式会社が公開しているner-wikipedia-dataset](https://github.com/stockmarkteam/ner-wikipedia-dataset)を利用させていただきます

In [ ]:
!wget "https://github.com/stockmarkteam/ner-wikipedia-dataset/raw/main/ner.json"

ダウンロードした学習データを確認してみましょう

In [ ]:
!head -15 ner.json

## 学習の実行
実際に学習を行っていきます

In [ ]:
model_output_dir = "./dest"
model_name = "cl-tohoku/bert-base-japanese-whole-word-masking"

### Tokenizerの準備
Tokenizerを用意します

In [ ]:
from transformers import BertJapaneseTokenizer

tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

なお、NEologdを使いたい場合など、TokenizerのMeCabにオプションを渡したい場合は[こちら](https://qiita.com/ken11_/items/fd20e69103bb0ce698af)を参考にしてください

### 学習データの前処理
先ほどダウンロードしてきた学習データを、学習に使えるように前処理していきます

In [ ]:
import noyaki
import json

def load_from_json(path: str) -> list:
    json_dict = json.load(open(path, "r"))
    features = []
    for unit in json_dict:
        tokenized_text = tokenizer.tokenize(unit["text"])
        spans = []
        for entity in unit["entities"]:
            span_list = []
            span_list.extend(entity["span"])
            span_list.append(entity["type"])
            spans.append(span_list)
        label = noyaki.convert(tokenized_text, spans, subword="##")
        features.append({"x": tokenized_text, "y": label})
    return features

In [ ]:
features = load_from_json("./ner.json")

featuresの中身を確認します

In [ ]:
print(features[:10])

学習データを `train`, `valid`, `test` に分割します

In [ ]:
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(features, test_size=0.2, random_state=123)
train_data, test_data = train_test_split(train_data, test_size=0.1, random_state=123)

### ラベル辞書の作成
ラベルの辞書を作成します  
これはあとでmodelのconfigに渡す情報となります

In [ ]:
def create_label_vocab(features: list) -> tuple:
    labels = [f["y"] for f in features]
    unique_labels = list(set(sum(labels, [])))
    label2id = {}
    for i, label in enumerate(unique_labels):
        label2id[label] = i
    id2label = {v: k for k, v in label2id.items()}
    return label2id, id2label

In [ ]:
label2id, id2label = create_label_vocab(features)
print(label2id)
print(id2label)

### モデルの準備
ベースモデルを用意します  
ここで先ほどの `label2id`, `id2label` を渡してあげることで、推論時のラベル復元が楽になります

In [ ]:
from transformers import BertForTokenClassification, BertConfig

config = BertConfig.from_pretrained(model_name, label2id=label2id, id2label=id2label)
model = BertForTokenClassification.from_pretrained(model_name, config=config)

In [ ]:
print(model)

### Trainerの準備
TrainingArgumentsを設定し、Trainerを作成していきます  
Trainerにはdata_collatorを渡してあげる必要があるので、data_collatorも作成します

data_collatorは[transformersにすでにあるもの](https://huggingface.co/docs/transformers/main_classes/data_collator)を利用することもできますが、ここでは自前で定義していきます

In [ ]:
import torch

def data_collator(features: list) -> dict:
    x = [f["x"] for f in features]
    y = [f["y"] for f in features]
    inputs = tokenizer(x, return_tensors=None, padding='max_length', truncation=True, max_length=64, is_split_into_words=True)
    input_labels = []
    for labels in y:
        pad_list = [-100] * 64
        for i, label in enumerate(labels):
            pad_list.insert(i, label2id[label])
        input_labels.append(pad_list[:64])
    inputs['labels'] = input_labels
    batch = {k: torch.tensor(v, dtype=torch.int64) for k, v in inputs.items()}
    return batch

ハイパーパラメータなどを定義しておきます

In [ ]:
ckpt_dir = "./ckpt"
batch_size = 8
epochs = 3
learning_rate = 3e-5
save_freq = 100

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(output_dir=ckpt_dir,
                         do_train=True,
                         do_eval=True,
                         do_predict=True,
                         per_device_train_batch_size=batch_size,
                         per_device_eval_batch_size=batch_size,
                         learning_rate=learning_rate,
                         num_train_epochs=epochs,
                         evaluation_strategy="steps",
                         eval_steps=save_freq,
                         save_strategy="steps",
                         save_steps=save_freq,
                         load_best_model_at_end=True,
                         remove_unused_columns=False,
                        )

In [ ]:
from transformers import Trainer, EarlyStoppingCallback

trainer = Trainer(model=model,
                  args=args,
                  data_collator=data_collator,
                  train_dataset=train_data,
                  eval_dataset=val_data,
                  callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
                 )

学習を実行します

In [ ]:
trainer.train()

できあがったモデルをテストします

In [ ]:
_, _, metrics = trainer.predict(test_data, metric_key_prefix="test")
print(metrics)

モデルをsaveします

In [ ]:
trainer.save_model(model_output_dir)

## モデルの検証
[seqeval](https://github.com/chakki-works/seqeval)を使って実際のモデル精度を検証していきます

### 推論用の関数を定義
学習したモデルを使って推論をするための関数を定義します

In [ ]:
import numpy as np

inference_model = BertForTokenClassification.from_pretrained(model_output_dir)
def inference(tokenized_text: list) -> list:
    inputs = tokenizer(tokenized_text, return_tensors="pt", padding='max_length', truncation=True, max_length=64, is_split_into_words=True)
    pred = inference_model(**inputs).logits[0]
    pred = np.argmax(pred.detach().numpy(), axis=-1)
    labels = []
    for i, label in enumerate(pred):
        if i + 1 > len(tokenized_text):
            continue
        labels.append(inference_model.config.id2label[label])
    return labels

正解データを用意します

In [ ]:
y_true = []
for unit in test_data:
    # 今回はmax_lengthを64にしているので正解データも切り詰めておく
    y_true.append(unit["y"][:64])
print(y_true)

同様に推論結果も用意します

In [ ]:
y_pred = []
for unit in test_data:
    y_pred.append(inference(unit["x"]))
print(y_pred)

### seqevalのclassification_reportを実行
seqevalのclassification_reportを使って検証します

In [ ]:
from seqeval.metrics import classification_report
from seqeval.scheme import BILOU

print(classification_report(y_true, y_pred, mode='strict', scheme=BILOU))

seqevalのstrictモードは厳密なので精度は低くなりがちです  
BILUOではstrictモードしかサポートされていないため、適宜BILUOをBIOに変換して使用するなど、タスクに合った精度検証を行ってください

## 推論
最後に、通常の推論用コードを紹介します

In [ ]:
def inference(text: str):
    model = BertForTokenClassification.from_pretrained(model_output_dir)
    tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)
    
    
    tokenized_text = tokenizer.tokenize(text)
    inputs = tokenizer(tokenized_text, return_tensors="pt", padding='max_length', truncation=True, max_length=64, is_split_into_words=True)
    pred = model(**inputs).logits[0]
    pred = np.argmax(pred.detach().numpy(), axis=-1)
    labels = []
    for i, label in enumerate(pred):
        if i + 1 > len(tokenized_text):
            continue
        labels.append(inference_model.config.id2label[label])
    print(tokenized_text)
    print(labels)

In [ ]:
print(inference("田中さんの会社の社長は鈴木さんです"))